**Course website**: http://www.leouieda.com/geofisica1

**Note**: This notebook is part of the course "Geofísica 1" of Geology program of the 
[Universidade do Estado do Rio de Janeiro](http://www.uerj.br/). 
All content can be freely used and adapted under the terms of the 
[Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)

# Prática 4 - Deconvolução de Euler

Objetivos:

* Explorar as soluções obtidas com a Deconvolução de Euler.
* Analisar os efeitos do ruído aleatório nas soluções obtidas.
* Adquirir a intuição da sensibilidade da Deconvolução aos parâmetros que a controlam.

## Preparação

Esse documento que você está usando é um [IPython notebook](http://ipython.org/notebook.html). É um documento interativo que mistura texto (como esse), código (como abaixo), e o resultado de executar o código (que pode ser números, texto, figuras, videos, etc). Esta prática usará a biblioteca [Fatiando a Terra](http://fatiando.org) de modelagem geofísica e também o módulo [numpy](http://www.numpy.org/).

O notebook é divido em células (como esta). Para editar o conteúdo de uma célula, clique nela (clique nesta para editar esse texto). Para executar uma célula, aperte `Shift + Enter`. Execute as duas células abaixo.

In [ ]:
%matplotlib inline
from __future__ import division
from IPython.html import widgets
import numpy as np
from fatiando.gravmag import prism, fourier, euler, sphere, polyprism
from fatiando import utils, gridder, mesher
from fatiando.vis import mpl, myv
import fatiando
mpl.rc('lines', linewidth=2)
mpl.rc('font', size=12)

In [ ]:
print('Versão do Fatiando a Terra: {}'.format(fatiando.__version__))

## Tarefa 1: Fonte simples (uma esfera)

Nessa tarefa vamos analisar os efeitos do erro aleatório nos resultados da deconvolução. Para isso vamos  usar um modelo simples com  uma única esfera.

Rode a célula abaixo para produzir a figura interativa.

In [ ]:
inc, dec = 45, 15
modelo = [mesher.Sphere(1000, 1000, 400, 300, {'magnetization': utils.ang2vec(5, inc, dec)})]
area = [0, 2000, 0, 2000]
shape = (100, 100)
x, y, z = gridder.regular(area, shape, z=0)
anomalia_pura = sphere.tf(x, y, z, modelo, inc, dec)
def fonte_simples(erro):
    n_janelas = 20 
    tamanho_janela = 600
    if erro > 0:
        anomalia = utils.contaminate(anomalia_pura, erro, seed=0)
    else:
        anomalia = anomalia_pura
    dx = fourier.derivx(x, y, anomalia, shape)
    dy = fourier.derivy(x, y, anomalia, shape)
    dz = fourier.derivz(x, y, anomalia, shape)
    prob = euler.Classic(x, y, z, anomalia, dx, dy, dz, 3)
    solver = euler.MovingWindow(prob, windows=(n_janelas, n_janelas), 
                                size=(tamanho_janela, tamanho_janela))
    solver.fit()    
    filtrado = [e for e in solver.estimate_ if e[2] > 0]
    # Graficos
    mpl.figure(figsize=(12, 4.5))
    mpl.subplot(1, 2, 1)
    mpl.title(u'Anomalia com erro de {} nT'.format(erro))
    mpl.axis('scaled')
    mpl.contourf(y, x, anomalia, shape, 20, cmap=mpl.cm.RdBu_r)
    if filtrado:
        x0, y0, z0 = np.transpose(filtrado)
        mpl.scatter(y0, x0, c=z0, s=80, cmap=mpl.cm.RdYlGn, vmin=0, vmax=1000)
        mpl.colorbar(pad=0).set_label('Profundidade (metros)')
    mpl.grid()
    mpl.m2km()
    mpl.subplot(1, 2, 2)
    mpl.title('Derivada em z')
    mpl.axis('scaled')
    mpl.contourf(y, x, dz, shape, 20, cmap=mpl.cm.RdBu_r)
    mpl.colorbar(pad=0).set_label('nT/m')
    mpl.square((0, tamanho_janela, 0, tamanho_janela), '--k', linewidth=2, label='Janela')
    mpl.legend(loc='upper right')
    mpl.m2km()
    mpl.tight_layout(pad=0)
    return filtrado
w = widgets.interactive(fonte_simples, 
                        erro=widgets.FloatSliderWidget(min=0, max=10, step=1, value=0))
w

### Sobre a figura

* A figura da esquerda mostra a anomalia magnética de campo total gerada pela esfera.
* O centro da esfera está a 400 metros de profundidade.
* Os círculos coloridos representam as soluções da deconvolução de Euler aplicada a anomalia.
* As cores representam a profundidade da solução (barra de cor).
* A figura da direita mostra a derivada da anomalia na direção z (vertical) calculada com a Transformada de Fourier.
* As derivadas em x, y e z são utilizadas na deconvolução. Somente a derivada em z é  mostrada aqui para facilitar.
* O quadrado preto tracejado representa a área da janela utilizada na deconvolução.
* O índice estrutural utilizado na deconvolução é 3.
* Somente 20% das  soluções são mantidas (as melhores). Soluções localizadas acima dos dados também são descartadas.

Rode a célula abaixo para **gerar uma figura 3D** com o modelo da esfera e as soluções da deconvolução de Euler.
* Essa figura 3D reflete os resultados  mostrados na figura acima. 
* Quando terminar, feche a janela da figura. Enquanto não fechar não poderá rodar mais nada aqui.
* Rode a célula de novo gerar a figura novamente.
* Se alterar a figura acima, a figura 3D será atualizada quando a gerar novamente.
* Para salvar uma foto da figura clicando no disquete.

In [ ]:
myv.figure()
for e in modelo:
    myv.points([[e.x, e.y, e.z]], color=(0, 0, 1), size=e.radius, opacity=0.3)
myv.points(w.result, size=30, color=(1, 0, 0))
bounds = area + [0, 1000]
myv.axes(myv.outline(bounds), ranges=[b*0.001 for b in bounds], fmt='%.1f')
myv.wall_bottom(bounds)
myv.wall_north(bounds)
myv.show()

### Tarefas e perguntas

1. Mantenha o erro em zero e gere a figura 3D.
    * Onde estão localizadas as soluções da deconvolução de Euler?
2. Aumente o erro. Se lembre de olhar tanto a figura em planta (o mapa) quanto o 3D.
    * Como o erro influencia nas soluções da deconvolução?
    * Por que a deconvolução é tão sensível ao erro?
    * A distribuição das soluções em 3D possui alguma semelhança com a forma do corpo (esfera)?    

## Tarefa 2: Múltiplas fontes (duas esferas)

Agora que vocês pegaram o jeito da deconvolução, vamos ver como  ela  se comporta para um modelo com duas esferas. Vamos analisar como as soluções são influenciadas pelo procedimento de dividir a área em janelas. Os parâmetros que controlam esse procedimento são o número de janelas e o seu tamanho.

Rode a célula abaixo para produzir a figura interativa.

In [ ]:
inc, dec = 45, 15
modelo = [mesher.Sphere(1500, 1500, 400, 300, {'magnetization': utils.ang2vec(5, inc, dec)}),
          mesher.Sphere(2500, 2500, 400, 300, {'magnetization': utils.ang2vec(5, -15, -20)})]
area = [0, 4000, 0, 4000]
shape = (100, 100)
x, y, z = gridder.regular(area, shape, z=0)
anomalia_pura = sphere.tf(x, y, z, modelo, inc, dec)
anomalia = utils.contaminate(anomalia_pura, 5, seed=0)
dx = fourier.derivx(x, y, anomalia, shape)
dy = fourier.derivy(x, y, anomalia, shape)
dz = fourier.derivz(x, y, anomalia, shape)
prob = euler.Classic(x, y, z, anomalia, dx, dy, dz, 3)
def duas_fontes(n_janelas, tamanho_janela):
    solver = euler.MovingWindow(prob, windows=(n_janelas, n_janelas), 
                                size=(tamanho_janela, tamanho_janela),
                                keep=0.15)
    solver.fit()    
    filtrado = [e for e in solver.estimate_ if e[2] > 0]
    mpl.figure(figsize=(7, 6))
    mpl.title(u'Deconvolução com {} x {} janelas de {} m'.format(n_janelas, n_janelas, tamanho_janela))
    mpl.axis('scaled')
    mpl.contourf(y, x, anomalia, shape, 20, cmap=mpl.cm.RdBu_r)
    if filtrado:
        x0, y0, z0 = np.transpose(filtrado)
        mpl.scatter(y0, x0, c=z0, s=80, cmap=mpl.cm.RdYlGn, vmin=0, vmax=1000)
        mpl.colorbar(pad=0).set_label('Profundidade (metros)')
    mpl.square((0, tamanho_janela, 0, tamanho_janela), '--k', linewidth=2, label='Janela')
    mpl.legend(loc='upper right')
    mpl.m2km()
    mpl.tight_layout(pad=0)
    return filtrado
w = widgets.interactive(duas_fontes, 
                        n_janelas=widgets.IntSliderWidget(min=5, max=50, step=5, value=30),
                        tamanho_janela=widgets.FloatSliderWidget(min=200, max=3000, step=100, value=1000))
w

### Sobre a figura

* A anomalia é gerada por duas esferas com magnetizações distintas.
* Os dados foram contaminados com erro de 5 nT.
* O quadrado preto tracejado representa a área da janela utilizada na deconvolução.
* O índice estrutural utilizado na deconvolução é 3.
* Somente 15% das  soluções são mantidas (as melhores). Soluções localizadas acima dos dados também são descartadas.
* O botão `n_janelas` controla o número de janelas em ambas as direções. `n_janelas = 10` significa que serão utilizadas 10 x 10 = 100  janelas para fazer a deconvolução. 
* O botão `tamanho_janela` controla o tamanho das janelas (em metros). `n_janelas = 10` e `tamanho_janela = 1000`  significa que a área será coberta por 10 x 10 = 100 janelas com 1000 x 1000 metros cada. Note que as janelas se  sobrepõe.

Rode a célula abaixo para **gerar uma figura 3D** com o modelo da esfera e as soluções da deconvolução de Euler.
* Essa figura 3D reflete os resultados  mostrados na figura acima. 
* Quando terminar, feche a janela da figura. Enquanto não fechar não poderá rodar mais nada aqui.
* Rode a célula de novo gerar a figura novamente.
* Se alterar a figura acima, a figura 3D será atualizada quando a gerar novamente.
* Para salvar uma foto da figura clicando no disquete.

In [ ]:
myv.figure()
for e in modelo:
    myv.points([[e.x, e.y, e.z]], color=(0, 0, 1), size=e.radius, opacity=0.3)
myv.points(w.result, size=30, color=(1, 0, 0))
bounds = area + [0, 1000]
myv.axes(myv.outline(bounds), ranges=[b*0.001 for b in bounds], fmt='%.1f')
myv.wall_bottom(bounds)
myv.wall_north(bounds)
myv.show()

### Tarefas e perguntas

1. Mantenha o tamanho das janelas em 1000 metros. Mude o número de janelas. Não se esqueça de olhar a figura 3D.
    * O número de soluções muda?
    * Como varia a dispersão das soluções obtidas?
2. Mantenha o número de janelas fixo em 30. Mude o tamanho das janelas.
    * O que acontece quando o tamanho das janelas é pequeno? Por que?
    * E quando o tamanho é grande? Por que?
    * Qual é o tamanho que produz as soluções mais concentradas em cada uma das 2 anomalias? 
    * Qual é a relação tamanho com as anomalias?

## Tarefa 3: Fontes não-ideais e o índice estrutural

Para complicar ainda mais, vamos fazer um teste com um modelo mais complexo: um sill, um batólito e um dique. Nesse exercício, vamos explorar a influência que o índice estrutural tem sobre as soluções.

Esse modelo é o que foi utilizado em:
[Uieda, Oliveira Jr., and Barbosa (2014), Geophysical tutorial: Euler deconvolution of potential-field data, The Leading Edge, 33(4), 448-450.](http://wiki.seg.org/wiki/Euler_deconvolution_of_potential_field_data_%28tutorial%29).


Rode a célula abaixo para produzir a figura interativa.

In [ ]:
area = [5000, 25000, 5000, 25000]
bounds = [0, 30000, 0, 30000, 0, 5000]
shape = (100, 100)
x, y, z = gridder.regular(area, shape, z=-300)
inc, dec = -15, 30
dike = mesher.PolygonalPrism(
    [[78.12500000000136, 17968.750000000004],
     [15156.250000000002, 19843.750000000004],
     [29843.75, 20781.250000000004],
     [29843.75, 21015.625000000004],
     [15078.125000000002, 20156.250000000004],
     [156.25000000000136, 18281.250000000004]], 0, 5000,
    props={'magnetization': utils.ang2vec(10, inc, dec)})
sill = mesher.PolygonalPrism(
    [[10000.000000000002, 8828.125000000004],
     [10156.250000000002, 9062.500000000004],
     [11328.125000000002, 10703.125000000004],
     [11875.000000000002, 11796.875000000004],
     [11953.125000000002, 12890.625000000004],
     [11406.250000000002, 13593.750000000004],
     [10156.250000000002, 13984.375000000004],
     [9375.000000000002, 13984.375000000004],
     [9218.750000000002, 13984.375000000004],
     [8437.500000000002, 12812.500000000004],
     [7968.750000000002, 11796.875000000004],
     [7968.750000000002, 10078.125000000004],
     [7968.750000000002, 9765.625000000004],
     [8593.750000000002, 9218.750000000004]], 1000, 1500,
    props={'magnetization': utils.ang2vec(10, inc, dec)})
batolito = mesher.PolygonalPrism(
    [[18046.875, 9921.875000000004],
     [19062.5, 10234.375000000004],
     [19765.625, 11093.750000000004],
     [20078.125, 12500.000000000004],
     [19843.75, 13515.625000000004],
     [18281.25, 14140.625000000004],
     [16640.625, 14140.625000000004],
     [15781.250000000002, 13671.875000000004],
     [15703.125000000002, 11718.750000000004],
     [15859.375000000002, 10390.625000000004],
     [16250.000000000002, 9843.750000000004]], 500, 4000,
    props={'magnetization': utils.ang2vec(2, inc, dec)})
modelo = [dike, sill, batolito]
anomalia = utils.contaminate(polyprism.tf(x, y, z, modelo, inc, dec), 5)
dx = fourier.derivx(x, y, anomalia, shape)
dy = fourier.derivy(x, y, anomalia, shape)
dz = fourier.derivz(x, y, anomalia, shape)
def complexo(indice):
    tamanho_janela = 3000
    prob = euler.Classic(x, y, z, anomalia, dx, dy, dz, indice)
    n_janelas = 50
    solver = euler.MovingWindow(prob, windows=(n_janelas, n_janelas), 
                                size=(tamanho_janela, tamanho_janela),
                                keep=0.3)
    solver.fit()    
    filtrado = [e for e in solver.estimate_ if e[2] > 0]
    mpl.figure(figsize=(7, 6))
    mpl.title(u'Índice estrutural {}'.format(indice))
    mpl.axis('scaled')
    mpl.contourf(y, x, anomalia, shape, 20, cmap=mpl.cm.RdBu_r)
    if filtrado:
        x0, y0, z0 = np.transpose(filtrado)
        mpl.scatter(y0, x0, c=z0, s=80, cmap=mpl.cm.RdYlGn, vmin=0, vmax=3000)
        mpl.colorbar(pad=0).set_label('Profundidade (metros)')
    mpl.square((5e3, 5e3 + tamanho_janela, 5e3, 5e3 + tamanho_janela), '--k', linewidth=2, label='Janela')
    mpl.legend(loc='upper left')
    for b in modelo:
        mpl.polygon(b, xy2ne=True)
    mpl.m2km()
    mpl.tight_layout(pad=0)
    return filtrado
w = widgets.interactive(complexo, 
                        indice=widgets.IntSliderWidget(min=1, max=4, step=1, value=3))
w

###  Sobre a figura

* A anomalia foi gerada pelos 3 corpos e contaminada com erro de 5 nT.
* As linhas pretas sólidas representam a projeção dos 3 corpos na superfície.
* Nenhum dos corpos aflora.
* Somente 30% das soluções são mantidas (as melhores). Soluções localizadas acima dos dados também são descartadas.
* O número de janelas é de 50 x 50 e o tamanho de 3000 x 3000 metros.
* O botão `indice` controla o índice estrutural.

### Índice estrutural

O índice estrutural (IS) representa o decaimento do campo magnético (ou gravitacional) com a distância. Esse fator só é definido para fontes ideias, como esferas, diques retos e verticais, etc, e varia para cada uma dessas fontes. Segundo Reid et al. (1990), para o caso magnético:

| Fonte             | IS  |
|-------------------|-----|
| Dique             |  1  |
| Sill              |  1  |
| Esfera            |  3  |
| Cilindro vertical |  2  |

**Referências**

Reid, A. B., J. M. Allsop, H. Granser, A. J. Millett, and I. W. Somerton (1990), Magnetic interpretation in three dimensions using Euler deconvolution, Geophysics, 55(1), 80–91, [doi:10.1190/1.1442774](http://dx.doi.org/doi:10.1190/1.1442774).

Rode a célula abaixo para **gerar uma figura 3D** com o modelo da esfera e as soluções da deconvolução de Euler.
* Essa figura 3D reflete os resultados  mostrados na figura acima. 
* Quando terminar, feche a janela da figura. Enquanto não fechar não poderá rodar mais nada aqui.
* Rode a célula de novo gerar a figura novamente.
* Se alterar a figura acima, a figura 3D será atualizada quando a gerar novamente.
* Para salvar uma foto da figura clicando no disquete.

In [ ]:
myv.figure()
myv.polyprisms(modelo, opacity=0.3)
myv.points(w.result, size=150, color=(1, 0, 0))
myv.axes(myv.outline(bounds), ranges=[b*0.001 for b in bounds], fmt='%.1f')
myv.wall_bottom(bounds)
myv.wall_north(bounds)
myv.show()

### Tarefas e perguntas

1. Mantenha o índice em 3 (esfera). Gere a figura 3D.
    * Qual dos 3 corpos  mais se aproxima de uma esfera?
    * As soluções da deconvolução estão localizadas no centro desse corpo, como acontecia com a esfera da Tarefa 1?
    * As soluções tem alguma semelhança com a forma do corpo (todo o volume, não só as bordas)?
    * Como são as soluções para os outros 2 corpos?
    * Elas acertam as profundidades desses 2 corpos?
2. Diminua o índice estrutural. Lembre de olhar a figura 3D.
    * Como variam as soluções (localização horizontal, profundidade, dispersão, etc)?
    * Qual seria o índice estrutural mais correto para o dique e para o sill?
    * Onde estão localizadas as soluções da deconvolução para esse índice?